In [2]:
# COMP257_A5 - Autoencoders
# Author: Young Park

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

# Load the data
from sklearn.datasets import fetch_olivetti_faces

faces = fetch_olivetti_faces()

# Split the training set, a validation set and a test set using a stratified sampling.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(faces.data, faces.target, test_size=0.2, random_state=13, stratify=faces.target)

In [3]:
X_train.shape

(320, 4096)

In [4]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# grid search with autoencoder
from scikeras.wrappers import KerasRegressor
from keras.optimizers import Adam
params = {
    'loss': ['mean_squared_error', 'binary_crossentropy', 'mean_absolute_error'],
    'optimizer': ['Adam(learning_rate=0.001)', 'Adam(learning_rate=0.01)', 'Adam(learning_rate=0.1)'],
}

def create_model(optimizer, loss):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(4096,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(4096, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss=loss)
    return model

model = KerasRegressor(build_fn=create_model(optimizer, loss), epochs=10, batch_size=32, verbose=0)

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=params, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, X_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

NameError: name 'optimizer' is not defined